In [4]:
! pip install -U googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import googlemaps 
import pandas as pd
import requests

In [21]:
import requests
import pandas as pd
import time

# Set the API endpoint
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set the parameters for the API request
params = {
    "location": "52.520008,13.404954",  # Berlin city center
    "radius": "20000",  # 20 km radius
    "type": "restaurant",
    "keyword": "food",  # Search for restaurants with the keyword "food"
    "key": "Your_API"  # Replace with your API key
}

# Initialize an empty DataFrame to store the results
restaurants_df = pd.DataFrame(columns=["Name", "Address", "Phone", "Website", "Rating"])

# Loop through the pages and extract the restaurant data
while len(restaurants_df) < 1000:
    # Make the API request
    response = requests.get(url, params=params)
    data = response.json()

    # Extract the restaurant data from the API response and append it to the DataFrame
    for restaurant in data["results"]:
        name = restaurant.get("name", "")
        address = restaurant.get("vicinity", "")
        phone = restaurant.get("formatted_phone_number", "")
        website = restaurant.get("website", "")
        rating = restaurant.get("rating", 0)
        restaurants_df = restaurants_df.append({"Name": name, "Address": address, "Phone": phone, "Website": website, "Rating": rating}, ignore_index=True)

        # Check if we have enough results
        if len(restaurants_df) >= 1000:
            break

    # Check if there are more results
    next_page_token = data.get("next_page_token", None)

    # Wait for a short time before making the next request
    time.sleep(2)

    # Set the next page token parameter
    while next_page_token is not None and len(restaurants_df) < 1000:
        params["pagetoken"] = next_page_token
        response = requests.get(url, params=params)
        data = response.json()
        next_page_token = data.get("next_page_token", None)
        
        # Extract the restaurant data from the API response and append it to the DataFrame
        for restaurant in data["results"]:
            name = restaurant.get("name", "")
            address = restaurant.get("vicinity", "")
            phone = restaurant.get("formatted_phone_number", "")
            website = restaurant.get("website", "")
            rating = restaurant.get("rating", 0)
            restaurants_df = restaurants_df.append({"Name": name, "Address": address, "Phone": phone, "Website": website, "Rating": rating}, ignore_index=True)
            
            # Check if we have enough results
            if len(restaurants_df) >= 1000:
                break
            
        # Wait for a short time before making the next request
        time.sleep(2)

# Drop any duplicate rows from the DataFrame
restaurants_df.drop_duplicates(inplace=True)

# Print the DataFrame
print(restaurants_df.head(1000))


                                                Name  \
0                                           Food Box   
1                                    The Fresh Seeds   
2                                        Eat Culture   
3                 Frittenwerk Berlin Friedrichstraße   
4                              China Food Restaurant   
..                                               ...   
56   W - Der Imbiss | vegan vegetarisches Restaurant   
57                                         Mr. Gyros   
58                                     Swing Kitchen   
59                                          Curry 61   
616                                     Risa Chicken   

                                               Address Phone Website Rating  
0                            Ganghoferstraße 1, Berlin                    5  
1                           Invalidenstraße 90, Berlin                  4.1  
2    Einkaufsbahnhof Alexanderplatz, Dircksenstraße...                  4.1  
3              

In [22]:
json_string = restaurants_df.to_json(orient='records')

In [23]:
json_string

'[{"Name":"Food Box","Address":"Ganghoferstra\\u00dfe 1, Berlin","Phone":"","Website":"","Rating":5},{"Name":"The Fresh Seeds","Address":"Invalidenstra\\u00dfe 90, Berlin","Phone":"","Website":"","Rating":4.1},{"Name":"Eat Culture","Address":"Einkaufsbahnhof Alexanderplatz, Dircksenstra\\u00dfe 2, Berlin","Phone":"","Website":"","Rating":4.1},{"Name":"Frittenwerk Berlin Friedrichstra\\u00dfe","Address":"Friedrichstra\\u00dfe 69, Berlin","Phone":"","Website":"","Rating":4},{"Name":"China Food Restaurant","Address":"Schlo\\u00dfstra\\u00dfe 91, Berlin","Phone":"","Website":"","Rating":4},{"Name":"Happy food","Address":"Danziger Str. 245, Berlin","Phone":"","Website":"","Rating":4},{"Name":"Curry and Burger","Address":"Hannah-Arendt-Stra\\u00dfe 1, Berlin","Phone":"","Website":"","Rating":4.3},{"Name":"Adam-Restaurant M\\u00fcllerstra\\u00dfe","Address":"M\\u00fcllerstra\\u00dfe 54, Berlin","Phone":"","Website":"","Rating":4.4},{"Name":"BurgerZone","Address":"Maa\\u00dfenstra\\u00dfe 2, B